In [20]:
# Z,X,M,Y are numpy arrays with 0/1 entries
from brute_force import bounds_PY_doX1

import numpy as np

Z = np.array([1, 0, 1, 0, 1, 0])  # binary treatment
X = np.array([1, 0, 0, 1, 0, 1])  # binary confounder
M = np.array([0, 1, 1, 0, 1, 0])  # binary mediator
Y = np.array([1, 0, 1, 0, 1, 0])  # binary outcome

lb, ub = bounds_PY_doX1(Z, X, M, Y)  # purely observational
# with experiments:
lb2, ub2 = bounds_PY_doX1(Z, X, M, Y)
print(lb2, ub2)
print(ub2-lb2)
#p_doZ=N.A, p_doM=N.A: 0.4999999999999999
#p_doZ=0.5, p_doM=N.A: 0.5
#p_doZ=N.A, p_doM=0.7: 0.5
#p_doZ=0.5, p_doM=0.7: 0.4999999999999999

0.16666667 0.6666666699999999
0.4999999999999999


In [ ]:
# Observational samples (binary; 0/1). Length = 60.
Z = np.array([0,0,0,1,1,1,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0], dtype=int)
X = np.array([0,0,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,1,0,0,1,0,0,1,0,0,1,0,0], dtype=int)
M = np.array([1,0,0,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0], dtype=int)
Y = np.array([0,0,0,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0], dtype=int)

# Interventional/experimental results you can feed as constraints
# p_doZ[z] = P(Y=1 | do(Z=1))
p_doZ = 0.568

# p_doM[m] = P(Y=1 | do(M=1))
p_doM = 0.8

lb, ub = bounds_PY_doX1(Z, X, M, Y, p_doM=p_doM, p_doZ=p_doZ)  # purely observational
print(ub-lb)
#none: 0.6206896529999999
#p_doM: 0.6206896580000001 (wider???)
#p_doZ: 0.4837241339999999
#both:  0.48372413400000003


0.48372413400000003
